# Credit Card Data Challenge

Ryan Reece    
Feb 15, 2018

## Inspect the data

In [1]:
import math
import pandas
import time
from sklearn.neighbors import NearestNeighbors

In [2]:
df_cc = pandas.read_csv('cc_info.csv')
df_cc.head(5)

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [3]:
df_tx = pandas.read_csv('transactions.csv')
df_tx.head(5)

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [4]:
print '%i cards, %i transactions' % (len(df_cc), len(df_tx))

984 cards, 294588 transactions


## 1. Find users never over limit

In [5]:
def get_limits(df):
    limits = dict()
    for index, row in df_cc.iterrows():
        limits[row['credit_card']] = row['credit_card_limit']
    return limits
    
def calculate_monthly_ballances(df):
    ballances = dict()
    for index, row in df_tx.iterrows():
        card = row['credit_card']
        date_str = row['date']
        date_obj = time.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        year = date_obj.tm_year
        month = date_obj.tm_mon
        amount = row['transaction_dollar_amount']
        ballances.setdefault((card, year, month), 0.0)
        ballances[(card, year, month)] += amount
    return ballances

def calculate_users_never_over_limit(ballances, limits):
    cards = limits.keys()
    cards.sort()
    good_cards = set(cards)
    for _k, _v in ballances.iteritems():
        card, year, month = _k
        ballance = _v
        limit = limits[card]
        if ballance > limit:
            good_cards.discard(card)
    good_cards = list(good_cards)
    good_cards.sort()
    return good_cards

In [6]:
limits = get_limits(df_cc)
cards = limits.keys()
cards.sort()
ballances = calculate_monthly_ballances(df_tx)
good_cards = calculate_users_never_over_limit(ballances, limits)

In [7]:
print 'There are %i cards that are never over limit.' % len(good_cards)

There are 862 cards that are never over limit.


In [8]:
print 'The first 5 cards are:'
print good_cards[:5]

The first 5 cards are:
[1003715054175576, 1013870087888817, 1023820165155391, 1073931538936472, 1077622576192810]


## 2. Find users over limit on a certain day

In [9]:
def calculate_ballances_for_day(df, year, month, day):
    ballances = dict()
    for index, row in df_tx.iterrows():
        card = row['credit_card']
        date_str = row['date']
        date_obj = time.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        _year = date_obj.tm_year
        _month = date_obj.tm_mon
        _day = date_obj.tm_mday
        if _year == year and _month == month and _day <= day:
            amount = row['transaction_dollar_amount']
            ballances.setdefault(card, 0.0)
            ballances[card] += amount
    return ballances

def calculate_users_over_limit(ballances, limits):
    cards = limits.keys()
    cards.sort()
    bad_cards = set()
    for _k, _v in ballances.iteritems():
        card = _k
        ballance = _v
        limit = limits[card]
        if ballance > limit:
            bad_cards.add(card)
    bad_cards = list(bad_cards)
    bad_cards.sort()
    return bad_cards

In [10]:
year = 2015
month = 10
day = 28
ballances = calculate_ballances_for_day(df_tx, year, month, day)
bad_cards = calculate_users_over_limit(ballances, limits)

In [11]:
print 'There are %i cards that are over limit on %i/%i/%i.' % (len(bad_cards), year, month, day)

There are 62 cards that are over limit on 2015/10/28.


You can make calls like above on every day to know the users over limit on that day.  If you want to know if a user goes over limit *on* that day, you could keep a set of users currently over limit and keep track of when a new users is added.

## 3. Anomaly detection

I use two different strategies:

1.  `find_anomalous_transactions_by_amount(df, card, n_sigma)` returns anomalies determined by calculating the mean and standard deviation amount and counting a transaction as anomalous if the amount is `n_sigma` standard deviations above the mean.  Currently using `n_sigma = 4`.

2.   `find_anomalous_transactions_by_location(df, card, n_neighbors, max_dist)` returns anomalies determined location. Uses nearest neighbors algorithm to find the n-th (`n_neighbors`) nearest neighbor and counts the transaction as anomalous if the distance in lat-long space is larger than `max_dist`.  Currently using `n_neighbors = 3` and `max_dist = 2.0`.

In [20]:
def find_anomalous_transactions_by_amount(df, card, n_sigma):
    count = 0
    avg   = 0.0
    std   = 0.0
    for index, row in df_tx.iterrows():
        _card = row['credit_card']
        if _card == card:
            amount = row['transaction_dollar_amount']
            count += 1
            avg   += amount
            std   += amount*amount
    avg = avg / count
    std = std / count
    std = std - avg*avg
    std = math.sqrt(std)
    anomalies = list()
    if count > 3:
        for index, row in df_tx.iterrows():
            _card = row['credit_card']
            if _card == card:
                amount = row['transaction_dollar_amount']
                if amount > n_sigma*std + avg:
                    date_str = row['date']
                    date_obj = time.strptime(date_str, '%Y-%m-%d %H:%M:%S')
                    _year = date_obj.tm_year
                    _month = date_obj.tm_mon
                    _day = date_obj.tm_mday
                    anomalies.append((card, year, month, day, amount, ['amount', avg, std]))
    return anomalies

def find_anomalous_transactions_by_location(df, card, n_neighbors, max_dist):
    locations = list()
    dates = list()
    for index, row in df_tx.iterrows():
        _card = row['credit_card']
        if _card == card:
            date_str = row['date']
            date_obj = time.strptime(date_str, '%Y-%m-%d %H:%M:%S')
            _year = date_obj.tm_year
            _month = date_obj.tm_mon
            _day = date_obj.tm_mday
            _lat = row['Lat']
            _long = row['Long']
            locations.append([_lat, _long])
            dates.append((_year, _month, _day))
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(locations)
    neighbor_distances, neighbor_indexes = nn.kneighbors(locations)
    anomalies = list()
    for i, dists in enumerate(neighbor_distances):
        assert len(dists) == n_neighbors
        dist = dists[-1] # distance to n-th nearest neighbor
        if dist > max_dist:
            neighbors = neighbor_indexes[i]
            year, month, day = dates[neighbors[0]]
            anomalies.append((card, year, month, day, dist, ['location', locations[neighbors[0]], locations[neighbors[1]]]))
    return anomalies

In [21]:
anomalies = find_anomalous_transactions_by_amount(df_tx, 1003715054175576, 4)

In [22]:
print '%i anomalies found.' % (len(anomalies))
print anomalies[:5]

6 anomalies found.
[(1003715054175576, 2015, 10, 28, 995.35, ['amount', 110.49747126436785, 128.87336593136322]), (1003715054175576, 2015, 10, 28, 818.47, ['amount', 110.49747126436785, 128.87336593136322]), (1003715054175576, 2015, 10, 28, 888.04, ['amount', 110.49747126436785, 128.87336593136322]), (1003715054175576, 2015, 10, 28, 859.31, ['amount', 110.49747126436785, 128.87336593136322]), (1003715054175576, 2015, 10, 28, 925.78, ['amount', 110.49747126436785, 128.87336593136322])]


In [23]:
anomalies = find_anomalous_transactions_by_location(df_tx, 1003715054175576, 3, 2.0)

In [24]:
print '%i anomalies found.' % (len(anomalies))
print anomalies[:5]

6 anomalies found.
[(1003715054175576, 2015, 9, 17, 38.603115390703429, ['location', [34.745014519984004, -2.83317873024418], [36.1881230051067, 8.360145344929201]]), (1003715054175576, 2015, 9, 18, 62.108195722395109, ['location', [25.1257488771669, 69.4752159457624], [24.659874274725198, 34.4292750818899]]), (1003715054175576, 2015, 9, 8, 35.049037221858875, ['location', [24.659874274725198, 34.4292750818899], [36.1881230051067, 8.360145344929201]]), (1003715054175576, 2015, 10, 25, 88.428319235814172, ['location', [-32.343578020098704, 102.03233765558399], [25.1257488771669, 69.4752159457624]]), (1003715054175576, 2015, 9, 25, 50.488328384175219, ['location', [-11.785998850623699, -7.37373323601339], [34.745014519984004, -2.83317873024418]])]


If I had more time, I would explore more how robust the anomaly detection by nearest-neighbors location is.  I realized I could also implement a simpler version that works like the `find_anomalous_transactions_by_amount`, calculating the mean and standard deviation of lat and long and having a threshold of anomalous deviations.

## Scratch

In [ ]:
"""
def calculate_card_spending_stats(df):
    avgs   = dict()
    stds   = dict()
    counts = dict()
    for index, row in df_tx.iterrows():
        card = row['credit_card']
        amount = row['transaction_dollar_amount']
        avgs.setdefault(card, 0.0)
        stds.setdefault(card, 0.0)
        counts.setdefault(card, 0.0)
        avgs[card]   += amount
        stds[card]   += amount*amount
        counts[card] += 1
    cards = avgs.keys()
    cards.sort()
    card_spending_stats = dict()
    for card in cards:
        avgs[card] = avgs[card] / counts[card]
        stds[card] = stds[card] / counts[card]
        stds[card] -= avgs[card]*avgs[card]
        stds[card] = math.sqrt(stds[card])
        card_spending_stats[card] = counts[card], avgs[card], stds[card]
    return card_spending_stats
"""

In [ ]:
"""
def find_anomalous_transactions_by_location(df, card, max_dist):
    count = 0
    avg_lat = 0.0
    avg_long = 0.0
    std   = 0.0
    for index, row in df_tx.iterrows():
        _card = row['credit_card']
        if _card == card:
            amount = row['transaction_dollar_amount']
            _lat   = row['Lat']
            _long  = row['Long']
            count += 1
            avg_lat += _lat
            avg_long+= _long
    avg = avg / count
    std = std / count
    std = std - avg*avg
    std = math.sqrt(std)
    anomalies = list()
    if count > 3:
        for index, row in df_tx.iterrows():
            _card = row['credit_card']
            if _card == card:
                amount = row['transaction_dollar_amount']
                if amount > n_sigma*std + avg:
                    date_str = row['date']
                    date_obj = time.strptime(date_str, '%Y-%m-%d %H:%M:%S')
                    _year = date_obj.tm_year
                    _month = date_obj.tm_mon
                    _day = date_obj.tm_mday
                    anomalies.append((card, year, month, day, amount, ['amount', avg, std]))
    return anomalies
"""